<a href="https://colab.research.google.com/github/DarkTHor55/learning-ml/blob/main/practice%202%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd



In [2]:
df=pd.read_csv("/content/used_cars.csv")

In [3]:
df.sample(20)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
2948,Volvo,XC90 T6 Inscription,2019,"60,000 mi.",Gasoline,316.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,None reported,Yes,"$37,000"
889,Chevrolet,Camaro 2LT,2015,"56,675 mi.",Gasoline,323.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Silver,Black,None reported,Yes,"$19,750"
2606,Chevrolet,Express 2500 Work Van,2017,"76,000 mi.",Gasoline,285.0HP 4.8L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,–,None reported,Yes,"$41,400"
3371,Porsche,Cayenne Turbo GT,2022,"1,766 mi.",Gasoline,4.0L,Automatic,Custom Color,Black,None reported,NaN,"$209,995"
3842,RAM,1500 Laramie,2022,"15,741 mi.",Gasoline,5.7L V8 16V MPFI OHV,8-Speed Automatic,Flame Red Clearcoat,Black,None reported,NaN,"$45,999"
946,Lincoln,Navigator Reserve,2022,"13,750 mi.",Gasoline,440.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,White,Beige,None reported,Yes,"$79,999"
3872,Ford,F-150 XLT,2016,"71,000 mi.",E85 Flex Fuel,385.0HP 5.0L 8 Cylinder Engine Flex Fuel Capab...,A/T,Black,–,At least 1 accident or damage reported,Yes,"$24,350"
336,BMW,540 i xDrive,2020,"37,426 mi.",Gasoline,2.0 Liter Turbo,Automatic,White,Black,None reported,NaN,"$39,998"
1053,Nissan,Armada Platinum,2018,"108,206 mi.",Gasoline,5.6 Liter,Automatic,White,Charcoal,At least 1 accident or damage reported,NaN,"$26,498"
3212,BMW,325 i,2003,"142,000 mi.",Gasoline,184.0HP 2.5L Straight 6 Cylinder Engine Gasoli...,5-Speed M/T,Silver,Black,At least 1 accident or damage reported,Yes,"$4,800"


In [4]:
df.shape

(4009, 12)

In [5]:
df.isnull().sum()

,0
brand,0
model,0
model_year,0
milage,0
fuel_type,170
engine,0
transmission,0
ext_col,0
int_col,0
accident,113


In [6]:
df['clean_title'] = df['clean_title'].fillna('No')

In [7]:
# Case 1: clean_title == 'No'
df.loc[
    df['accident'].isna() & (df['clean_title'] == 'No'),
    'accident'
] = 'None reported'

# Case 2: clean_title != 'No'
df.loc[
    df['accident'].isna() & (df['clean_title'] != 'No'),
    'accident'
] = 'At least 1 accident or damage reported'


In [8]:
df['fuel_type'] = df['fuel_type'].fillna('Unknown')
df['fuel_type'] = df['fuel_type'].replace(['–', 'not supported'], 'Unknown')

In [9]:
df['milage'] = (
    df['milage']
    .str.replace(r'[^0-9]', '', regex=True)
    .astype(int)
)

In [10]:
df['price'] = (
    df['price']
    .str.replace(r'[^0-9]', '', regex=True)
    .astype(int)
)

In [11]:
def merge_rare(df, col, threshold):
    freq = df[col].value_counts()
    df[col] = df[col].replace(freq[freq <= threshold].index, 'Others')

merge_rare(df, 'int_col', 150)
merge_rare(df, 'ext_col', 200)
merge_rare(df, 'transmission', 200)
merge_rare(df, 'engine', 40)
merge_rare(df, 'model', 15)
merge_rare(df, 'brand', 100)


In [12]:
from sklearn.model_selection import train_test_split

X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [13]:
cat_cols = [
    'clean_title','accident','fuel_type','brand',
    'model','engine','transmission','ext_col','int_col'
]

num_cols = ['model_year', 'milage']


In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_cols),
        ('num', StandardScaler(), num_cols)
    ]
)


In [15]:
X_train_final = preprocessor.fit_transform(X_train)
X_test_final  = preprocessor.transform(X_test)


In [16]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train_final, y_train)


LinearRegression()

In [17]:
from sklearn.metrics import mean_absolute_error, r2_score

y_pred = model.predict(X_test_final)

mae = mean_absolute_error(y_test, y_pred)
r2  = r2_score(y_test, y_pred)

print("MAE:", mae)
print("R2 :", r2)


MAE: 30075.51160390762
R2 : 0.038165391078998634


In [136]:
df['fuel_type'] = df['fuel_type'].fillna('Unknown')


In [137]:
df['fuel_type'] = df['fuel_type'].replace(
    ['–', 'not supported'],
    'Unknown'
)


In [138]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('price', axis=1), df['price'], test_size=0.2, random_state=42)

In [139]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(drop='first', handle_unknown='ignore')


In [140]:
df['fuel_type'].value_counts()

,count
fuel_type,
Gasoline,3309
Unknown,217
Hybrid,194
E85 Flex Fuel,139
Diesel,116
Plug-In Hybrid,34


In [141]:
df['brand'].value_counts()

,count
brand,
Ford,386
BMW,375
Mercedes-Benz,315
Chevrolet,292
Porsche,201
Audi,200
Toyota,199
Lexus,163
Jeep,143


In [142]:
df['model'].value_counts()

,count
model,
M3 Base,30
F-150 XLT,24
Corvette Base,22
1500 Laramie,18
Camaro 2SS,17
...,...
Mazda6 Signature,1
SLK-Class SLK320,1
Protege DX,1


In [143]:
df['engine'].value_counts()

,count
engine,
2.0L I4 16V GDI DOHC Turbo,52
355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel,48
420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,47
–,45
300.0HP 3.0L Straight 6 Cylinder Engine Gasoline Fuel,44
...,...
305.0HP 6.8L 10 Cylinder Engine Gasoline Fuel,1
562.0HP 5.2L 10 Cylinder Engine Gasoline Fuel,1
245.0HP 5.9L Straight 6 Cylinder Engine Diesel Fuel,1


In [171]:
# milage
df['milage'] = (
    df['milage']
    .str.replace(',', '', regex=False)
    .str.replace(' mi.', '', regex=False)
    .astype(int)
)

# price
df['price'] = (
    df['price']
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(int)
)


AttributeError: Can only use .str accessor with string values!

In [145]:
df['milage'].dtype


dtype('int64')

In [146]:
df['transmission'].value_counts()

,count
transmission,
A/T,1037
8-Speed A/T,406
Transmission w/Dual Shift Mode,398
6-Speed A/T,362
6-Speed M/T,248
...,...
10-Speed Automatic with Overdrive,1
9-Speed Automatic with Auto-Shift,1
SCHEDULED FOR OR IN PRODUCTION,1


In [147]:
df['ext_col'].value_counts()

,count
ext_col,
Black,905
White,816
Gray,496
Silver,374
Blue,349
...,...
Pristine White,1
Garnet Red Metallic,1
Cayenne Red Tintcoat,1


In [148]:
df['int_col'].value_counts()

,count
int_col,
Black,2025
Beige,538
Gray,472
Brown,157
–,133
...,...
Gray w/Blue Bolsters,1
Very Light Cashmere,1
Black / Gray,1


In [149]:
df['accident'].value_counts()

,count
accident,
None reported,3023
At least 1 accident or damage reported,986


In [150]:
df['clean_title'].value_counts()

,count
clean_title,
Yes,3413
No,596


In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   int64 
 4   fuel_type     4009 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      4009 non-null   object
 10  clean_title   4009 non-null   object
 11  price         4009 non-null   object
dtypes: int64(2), object(10)
memory usage: 376.0+ KB


In [152]:
df['price'] = (
    df['price']
    .str.replace('$', '', regex=False)   # dollar sign remove
    .str.replace(',', '', regex=False)   # comma remove
    .astype(int)
)

categories modification


In [153]:
freq = df['int_col'].value_counts()
df['int_col'] = df['int_col'].replace( freq[freq <= 150].index, 'Others')

In [154]:
freq = df['ext_col'].value_counts()
df['ext_col'] = df['ext_col'].replace( freq[freq <= 200].index, 'Others')

In [155]:
freq = df['transmission'].value_counts()
df['transmission'] = df['transmission'].replace( freq[freq <= 200].index, 'Others')

In [156]:
freq = df['engine'].value_counts()
df['engine'] = df['engine'].replace( freq[freq <= 40].index, 'Others')

In [157]:
freq = df['model'].value_counts()
df['model'] = df['model'].replace( freq[freq <= 15].index, 'Others')

In [158]:
freq = df['brand'].value_counts()
df['brand'] = df['brand'].replace( freq[freq <= 100].index, 'Others')

In [160]:
X_train_ohe = ohe.fit_transform(X_train[cat_cols])
X_test_ohe  = ohe.transform(X_test[cat_cols])


/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [3, 4, 5, 6, 7, 8] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [169]:
import numpy as np

X_train_ohe = X_train_ohe.reshape(-1, 1)
X_test_ohe  = X_test_ohe.reshape(-1, 1)
X_train_new=np.hstack((X_train[['model_year','milage']].values,X_train_ohe))
X_test_new=np.hstack((X_test[['model_year','milage']].values,X_test_ohe))

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [167]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)
X_test_scaled = scaler.fit_transform(X_test_ohe)
X_train_scaled = scaler.transform(X_train_ohe)

In [170]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   int64 
 4   fuel_type     4009 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      4009 non-null   object
 10  clean_title   4009 non-null   object
 11  price         4009 non-null   int64 
dtypes: int64(3), object(9)
memory usage: 376.0+ KB
